In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
pd.set_option('display.max_columns', None, "display.max_rows", None)

In [2]:
houses = pd.read_csv('/Users/jaycohen/Documents/NYCDSA/Machine_Learning_Housing_Project/HousingML/train05-23.csv')

In [3]:
houses["ExterQual"] = houses["ExterQual"].apply(lambda x: 2 if x == "Po" else 4 if x == "Fa" else 6 if x == "TA" else 8 if x == "Gd" else 10)
houses["KitchenQual"] = houses["KitchenQual"].apply(lambda x: 0 if x == False else 2 if x == "Po" else 4 if x == "Fa" else 6 if x == "TA" else 8 if x == "Gd" else 10)
houses["TotalBsmtSF"] = houses["TotalBsmtSF"].apply(lambda x: 1 if x == 0 else x)
houses["GarageArea"] = houses["GarageArea"].apply(lambda x: 1 if x == 0 else x)
houses["TotalBsmtSF"] = houses["TotalBsmtSF"].fillna(value=1)
houses["GarageArea"] = houses["GarageArea"].fillna(value=1)

In [4]:
houses['log_SalePrice']=np.log(houses['SalePrice'])
houses['log_LotArea']=np.log(houses['LotArea'])
houses['log_GrLivArea']=np.log(houses['GrLivArea'])
houses['log_GarageArea']=np.log(houses['GarageArea'])
houses['log_TotalBsmtSF']=np.log(houses['TotalBsmtSF'])

In [17]:
houses = houses.drop([523,1298] ,axis=0)

In [5]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
ols = linear_model.LinearRegression() 
from sklearn.model_selection import KFold, cross_val_score
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Ridge, RidgeCV

In [6]:
featurestoselect = houses[['log_GrLivArea', 'log_LotArea', 'log_TotalBsmtSF', 'log_GarageArea', 'OverallQual', 'ExterQual', 'KitchenQual', 'YrSold', 'YearBuilt']]
targetlogsaleprice = houses.log_SalePrice

In [7]:
X = pd.get_dummies(featurestoselect, drop_first=True, dummy_na=True)
y = targetlogsaleprice
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
ols.fit(X_train, y_train)
ols.score(X_train, y_train)

0.8457530275761557

In [9]:
ols.score(X_test, y_test)

0.8419312210490425

In [23]:
coeff_df = pd.DataFrame(ols.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
log_GrLivArea,0.393874
log_LotArea,0.145287
log_TotalBsmtSF,0.029652
log_GarageArea,0.017314
OverallQual,0.083565
ExterQual,0.020214
KitchenQual,0.042584
YrSold,-0.000267
YearBuilt,0.002497


In [24]:
testdata = pd.read_csv('/Users/jaycohen/Documents/NYCDSA/Machine_Learning_Housing_Project/HousingML/data/test.csv')

In [25]:
testdata["ExterQual"] = testdata["ExterQual"].apply(lambda x: 2 if x == "Po" else 4 if x == "Fa" else 6 if x == "TA" else 8 if x == "Gd" else 10)
testdata["KitchenQual"] = testdata["KitchenQual"].apply(lambda x: 0 if x == False else 2 if x == "Po" else 4 if x == "Fa" else 6 if x == "TA" else 8 if x == "Gd" else 10)
testdata["TotalBsmtSF"] = testdata["TotalBsmtSF"].apply(lambda x: 1 if x == 0 else x)
testdata["GarageArea"] = testdata["GarageArea"].apply(lambda x: 1 if x == 0 else x)
testdata["TotalBsmtSF"] = testdata["TotalBsmtSF"].fillna(value=1)
testdata["GarageArea"] = testdata["GarageArea"].fillna(value=1)

In [26]:
testdata['log_LotArea']=np.log(testdata['LotArea'])
testdata['log_GrLivArea']=np.log(testdata['GrLivArea'])
testdata['log_GarageArea']=np.log(testdata['GarageArea'])
testdata['log_TotalBsmtSF']=np.log(testdata['TotalBsmtSF'])

In [27]:
for col in ['SaleType']:
    testdata[col] = testdata[col].astype('category')

In [28]:
testdata2 = testdata[['log_GrLivArea', 'log_LotArea', 'log_TotalBsmtSF', 'log_GarageArea', 'OverallQual', 'ExterQual', 'KitchenQual', 'YrSold', 'YearBuilt']]

In [29]:
testdata2.columns[testdata2.isna().any()].tolist()

[]

In [114]:
logSalePrice_pred = ols.predict(testdata2)

In [115]:
logSubmission = pd.DataFrame({'Id': testdata['Id'], 'LogSalePrice': logSalePrice_pred})

In [116]:
logSubmission.head(25)

,Id,LogSalePrice
0,1461,11.942187
1,1462,12.191214
2,1463,12.274269
3,1464,12.315820
4,1465,12.312582
5,1466,12.307917
6,1467,12.162165
7,1468,12.250561
8,1469,12.331127
9,1470,11.820030


In [119]:
logSubmission['SalePrice'] = np.exp(logSubmission['LogSalePrice'])

In [120]:
logSubmission.head(25)

,Id,LogSalePrice,SalePrice
0,1461,11.942187,153612.267090
1,1462,12.191214,197050.168345
2,1463,12.274269,214115.066727
3,1464,12.315820,223199.232586
4,1465,12.312582,222477.668599
5,1466,12.307917,221442.330865
6,1467,12.162165,191408.474938
7,1468,12.250561,209098.586326
8,1469,12.331127,226641.961623
9,1470,11.820030,135948.362808


In [121]:
SubmissionDF = logSubmission[['Id','SalePrice']]

In [122]:
SubmissionDF.to_csv(r'/Users/jaycohen/Documents/NYCDSA/Machine_Learning_Housing_Project/HousingML/Jay/Submission1DF.csv', index = False)